# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,weno,7.4515,151.8468,30.97,70,100,4.63,FM,1701663472
1,1,isafjordur,66.0755,-23.1240,0.89,85,100,3.96,IS,1701663472
2,2,edinburgh of the seven seas,-37.0676,-12.3116,14.59,85,2,5.22,SH,1701663472
3,3,ariquemes,-9.9133,-63.0408,24.62,84,32,0.88,BR,1701663472
4,4,olonkinbyen,70.9221,-8.7187,0.20,85,36,6.03,SJ,1701663472


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map1 = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles='EsriNatGeo', frame_width=700, frame_height=500,
    color="Humidity", size="Humidity", alpha=0.7,
    hover_cols=["City", "Country", "Humidity"]
)
# Display the map
# YOUR CODE HERE
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_conditions = (city_data_df["Max Temp(C)"] < 25) & (city_data_df["Max Temp(C)"] > 18) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] < 50)
ideal_cities_df = city_data_df[ideal_conditions]

# Drop any rows with null values
# YOUR CODE HERE
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_cities_df.sample(10)

,City_ID,City,Lat,Lng,Max Temp(C),Humidity,Cloudiness,Wind Speed,Country,Date
21,21,taoudenni,22.6783,-3.9836,21.96,39,0,3.69,ML,1701663475
481,481,zouerate,22.7187,-12.4521,18.49,53,8,3.52,MR,1701663549
101,101,al wajh,26.2455,36.4525,21.97,68,0,3.09,SA,1701663489
82,82,quelimane,-17.8786,36.8883,19.97,100,20,1.54,MZ,1701663485
102,102,riachao do jacuipe,-11.8069,-39.3856,22.38,89,2,1.82,BR,1701663294
582,582,itapuranga,-15.5622,-49.9486,20.08,98,38,1.99,BR,1701663567
46,46,nelson bay,-32.7167,152.1500,24.49,61,23,2.68,AU,1701663478
373,373,nova vicosa,-17.8919,-39.3719,24.82,86,5,4.63,BR,1701663533
109,109,al ghayzah,16.2079,52.1760,24.16,59,13,2.48,YE,1701663490
111,111,cabo san lucas,22.8909,-109.9124,23.62,63,0,2.73,MX,1701663490


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
290,palmas,BR,-10.2128,-48.3603,88,
82,quelimane,MZ,-17.8786,36.8883,100,
201,thompson,CA,55.7435,-97.8558,93,
236,sur,OM,22.5667,59.5289,51,
191,selfoss,IS,63.9331,-20.9971,58,
112,inarajan village,GU,13.2736,144.7484,82,
330,east ballina,AU,-28.8667,153.5833,72,
219,dingwall,GB,57.5953,-4.4272,96,
307,humaita,BR,-7.5061,-63.0208,88,
89,merei,RO,45.1333,26.6833,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
weno - nearest hotel: High Tide Hotel
isafjordur - nearest hotel: Hótel Horn
edinburgh of the seven seas - nearest hotel: No hotel found
ariquemes - nearest hotel: Aquarius
olonkinbyen - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
atafu village - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
stanley - nearest hotel: Hotel 52
serhetabat - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
adamstown - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
bilibino - nearest hotel: No hotel found
coquimbo - nearest hotel: Hotel Iberia
gadzhiyevo - nearest hotel: Оазис
blackmans bay - nearest hotel: Villa Howden
tanumah - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
magadan - nearest hotel: Гостиница «ВМ-Центр»
taoudenni - nearest hotel: No hotel found
aasiaat - nearest hotel

selfoss - nearest hotel: Miðgarður
codrington - nearest hotel: No hotel found
severnoye - nearest hotel: No hotel found
bechar - nearest hotel: Hôtel Madania
el abiodh sidi cheikh - nearest hotel: No hotel found
quixada - nearest hotel: Hotel Monólitos
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
bardai - nearest hotel: No hotel found
anadyr - nearest hotel: Гостевой дом
hithadhoo - nearest hotel: Pebbles Inn
thompson - nearest hotel: Thompson Inn
dudinka - nearest hotel: Виктория
margate - nearest hotel: The Reading Rooms
ayr - nearest hotel: The Berkeley
tofol - nearest hotel: Nautilus Resort
san antonio de pale - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
akdepe - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
abadan - nearest hotel: خانه معلم
whitehorse - nearest hotel: Town & Mountain Hotel
celestun - nearest hotel: Hotel Flamingo Playa
berdigestyakh - nearest hotel: No hotel found
santa b

juegang - nearest hotel: No hotel found
namibe - nearest hotel: Hotel Chik Chik Namibe
paphos - nearest hotel: Mayfair Hotel
paracatu - nearest hotel: Hotel Presidente
bandarbeyla - nearest hotel: No hotel found
ouesso - nearest hotel: Auberge Nianina
ifanadiana - nearest hotel: Win
puerto santa - nearest hotel: No hotel found
masterton - nearest hotel: Copthorne Hotel Solway Park
harstad - nearest hotel: Scandic Harstad
lorengau - nearest hotel: Seeadler Bay Hotel
badarpur - nearest hotel: Meera Lodge
ocean city - nearest hotel: Watson's Regency Suites
krotovka - nearest hotel: Гостиница спорткомплекса
santo antonio - nearest hotel: Pousada Onça Puma
lesosibirsk - nearest hotel: No hotel found
kitimat - nearest hotel: MStar Hotel
baise city - nearest hotel: 梦之源大酒店
viedma - nearest hotel: Níjar
tolanaro - nearest hotel: Hôtel Mahavokey
puerto baquerizo moreno - nearest hotel: Dory's House
egletons - nearest hotel: Hôtel Ibis
alice springs - nearest hotel: Aurora Alice Springs
saint-pau

ghat - nearest hotel: فندق تاسيلي
ta`u - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
piensk - nearest hotel: Pałac w Lasowie
dimitrovgrad - nearest hotel: Черемшан
chibougamau - nearest hotel: No hotel found
itapuranga - nearest hotel: No hotel found
kenai - nearest hotel: Beluga Lookout Hotel
pirapozinho - nearest hotel: Hotel Almanara
la paz - nearest hotel: Alojamiento Tokio
tahe - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,weno,FM,7.4515,151.8468,70,High Tide Hotel
1,isafjordur,IS,66.0755,-23.1240,85,Hótel Horn
2,edinburgh of the seven seas,SH,-37.0676,-12.3116,85,No hotel found
3,ariquemes,BR,-9.9133,-63.0408,84,Aquarius
4,olonkinbyen,SJ,70.9221,-8.7187,85,No hotel found
...,...,...,...,...,...,...
582,itapuranga,BR,-15.5622,-49.9486,98,No hotel found
583,kenai,US,60.5544,-151.2583,82,Beluga Lookout Hotel
584,pirapozinho,BR,-22.2753,-51.5000,100,Hotel Almanara
585,la paz,BO,-16.5000,-68.1500,87,Alojamiento Tokio


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_with_hotels = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles='EsriNatGeo', frame_width=700, frame_height=500,
    color="Humidity", size="Humidity", alpha=0.7,
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)

# Display the map
map_with_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)